In [1]:
import os
import openai
import demoji
import pandas as pd

from time import sleep
from tqdm.notebook import tqdm
from unidecode import unidecode


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
path = "/home/crarojasca/Monash/MDFI_misinformation/Twitter_scrapped/climateScam/"
files = os.listdir(path)
contrarian_data = pd.DataFrame()

### Climate Scam Scrapped data
for file in files:
    tmp = pd.read_csv(path + file)
    contrarian_data = pd.concat([contrarian_data, tmp])

## Filter original tweets
contrarian_data = contrarian_data[contrarian_data.referenced_tweets.isna()].copy(deep=True)
contrarian_data = contrarian_data[["text", "id"]]#.drop("Unnamed: 0", axis=1)
contrarian_data["dataset"] = "climatescam"

## Join CARDS Waterloo Dataset
tmp = pd.read_csv("datasets/cards_waterloo.csv", low_memory=False)
tmp = tmp[(tmp.labels==1)&(tmp.PARTITION=="TRAIN")][["prompt", "tweetid", "DATASET"]].rename(
    columns={"prompt":"text", "tweetid":"id", "DATASET":"dataset"})
contrarian_data = pd.concat([contrarian_data, tmp])

In [3]:
contrarian_data.dataset.value_counts()

cards          7134
climatescam    6289
waterloo       3192
Name: dataset, dtype: int64

From this contrarian climate change claims generate a new one:

well done!

#firefauci
#neverforgetthesepeople
#ClimateScam
#jimcramer
#endthefed
#endthemandates
#trudeaumustgo https://t.co/0GvDbhzP9u

@TechInsider He wrote himself a pardon just like the catholic popes in the 15th century! #ClimateScam

Democrats have wasted billions.
#Democrats #ClimateScam
https://t.co/q0TfcGwSLK

Another tweet from a non scientist who claims to know EVERYTHING and expects you to worship at her alter of garbage. #ClimateScam https://t.co/lojpM2tH1C

Your AFTERNOON REMINDER that Indiana Jones 5 might have a climate change lecture involved so #BoycottIndianaJones5 #ClimateScam

**The green energy industry has been manipulating data to support their agenda. #ClimateScam #GreenEnergyScam**

In [4]:
FILE = "datasets/generated_disinformation.csv"

In [5]:
def generate_prompt(texts, nshots=3):
    """Generates"""
    samples = texts.sample(nshots).tolist()
    samples = ["{}. {}".format(
        i+1, unidecode(sample.replace("\n", ""))) for i, sample in enumerate(samples)]
    fewshots = "\n\n".join(samples) + "\n\n4. "
    query = "From these contrarian climate change claims generate a new one:\n\n"
    prompt = query + fewshots
    
    return prompt

texts = contrarian_data["text"]
n = 1200
batch = 20

if os.path.isfile(FILE):
    completions = pd.read_csv(FILE)["text"].tolist()
else:
    completions = []

for i in tqdm(range(n//batch)):
    sleep(60)

    prompts = [generate_prompt(texts) for i in range(batch)]
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompts,
      temperature=0,
      max_tokens=60,
    )
    completions += [r["text"] for r in response["choices"]]

    if i%50==0:
        data = pd.DataFrame(completions, columns=["text"])
        data.to_csv(FILE)

  0%|          | 0/60 [00:00<?, ?it/s]